# Basic dynamic model

In [1]:
DATA_PATH = '../backend/data/dynamic_dataset'


In [2]:
#Load DATA
from sign.training.landmark_extraction.MediaPiper import MediaPiper

mp = MediaPiper()
gestures = mp.process_dynamic_gestures_from_folder(DATA_PATH)

2024-03-12 10:58:40.915888: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 10:58:40.921456: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 10:58:40.978826: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 10:58:42.749753: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I0000 00:00:1710237528.480553   56699 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1710237528.485919   56778 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) UHD Graphics 620 (WHL GT2)
INFO: Created 

[DynamicGesture(label='fist-up', results=[[MediapipeResult(multi_hand_landmarks=[MediapipeLandmark(x=0.34373682737350464, y=0.5757182240486145, z=-2.6760096716316184e-07), MediapipeLandmark(x=0.3102372884750366, y=0.5407504439353943, z=0.005808718968182802), MediapipeLandmark(x=0.303884357213974, y=0.5024613738059998, z=0.003782500047236681), MediapipeLandmark(x=0.31567782163619995, y=0.47917449474334717, z=0.00013380570453591645), MediapipeLandmark(x=0.33413976430892944, y=0.47257956862449646, z=-0.003005580510944128), MediapipeLandmark(x=0.29940176010131836, y=0.4680458903312683, z=-0.005047192331403494), MediapipeLandmark(x=0.31824418902397156, y=0.4675712287425995, z=-0.009483854286372662), MediapipeLandmark(x=0.3254202902317047, y=0.4885413646697998, z=-0.011896930634975433), MediapipeLandmark(x=0.32553258538246155, y=0.5055155158042908, z=-0.013701613992452621), MediapipeLandmark(x=0.3237922787666321, y=0.4635227918624878, z=-0.011416434310376644), MediapipeLandmark(x=0.340716421

In [5]:
[gesture.label for gesture in gestures]

['fist-up', 'fist-down']

In [5]:
import numpy as np
from sign.trajectory import TrajectoryBuilder, trajectory

bob = TrajectoryBuilder()

gesture_trajector_map: dict[str, list[trajectory]] = {}

for gesture in gestures:
    gesture_trajector_map[gesture.label] = []
    for sequence in gesture.results:
        new_sequence = []
        for image_mp_res in sequence:
            hand_landmarks = np.array(image_mp_res.multi_hand_landmarks)
            new_sequence.append(hand_landmarks)
        sequence_as_np_array = np.array(new_sequence)
        sequence_trajectory = bob.make_trajectory(sequence_as_np_array)

        gesture_trajector_map[gesture.label].append(sequence_trajectory)


print(gesture_trajector_map)

🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥
{'fist-up': [trajectory(directions=[trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.UP: 1>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.DOWN: -1>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.UP: 1>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_e

In [7]:
X = []
y = []

for k,gesture in gesture_trajector_map.items():
    min = len(gesture[0].directions)
    for trajectory in gesture[1:]:
        length = len(trajectory.directions)
        if min > length:
            min = length
    for trajectory in gesture:
        if len(trajectory.directions) > min:
            for i in range(len(trajectory.directions) - min):
                trajectory.directions.pop()


    for trajector in gesture:
        xyz  = []
        for te in trajector.directions:
            xyz.append(te.x.value)
            xyz.append(te.y.value)
            xyz.append(te.z.value)
        X.append(np.array(xyz))
        y.append(k)

X = np.array(X)
y = np.array(y)
X

array([[ 0, -1,  1,  0, -1, -1,  0, -1,  1,  0, -1,  0,  0, -1,  0,  0,
        -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,
        -1,  0, -1,  0],
       [ 0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0,
        -1,  1,  0, -1, -1,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,
         0,  0, -1,  0],
       [ 0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0,
        -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,  0,  0, -1,
         0,  0, -1,  0],
       [-1,  1,  1,  0,  1,  1,  0,  1,  0,  0,  1,  0,  0,  1,  0,  0,
         1,  0,  0,  1, -1, -1,  1,  0,  0,  1,  0,  0,  1, -1,  0,  1,
         0,  0,  1, -1],
       [ 0,  1,  0,  0,  1,  0,  0,  1,  0,  0,  1,  0,  0,  1,  0,  0,
         1,  1,  0,  1,  0,  0,  1, -1,  0,  1, -1,  0,  1,  0, -1,  1,
         0,  0,  1,  0],
       [ 0,  1,  0,  0,  1,  0,  0,  1,  0,  0,  1,  0,  0,  1,  0,  0,
         1,  0, -1,  1,  0, -1,  1,  0, -1,  1,  0, -1,  1,  0,  0,  1,
         0,

In [8]:
from sklearn.linear_model import LinearRegression, LogisticRegression

model = LogisticRegression()
model.fit(X,y)

LogisticRegression()

In [9]:
from joblib import dump
dump(model, 'dynamic_model.joblib')

['dynamic_model.joblib']